In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, save_string_to_file, get_extreme_values, create_directory_structure, save_table, save_report
# settings
region_column_name = 'Region'
table_name = 'dnssec_by_region'
report_name = 'dnssec_by_region'
category = 'dnssec'
column_name_to_results_global = 'Global #'
create_directory_structure()

source_df = load_data('dnssec_checker')

In [2]:
# sanity dataset
columns_names = ['has_dnssec', 'dnssec_is_valid']
for column in columns_names:
    source_df.loc[source_df[column].isna(), column] = False

In [3]:
# Analyze of HEIs with DNSSEC enabled by region

# settings
column_to_sort = 'Without DNSSEC (Public) %'
sort_ascending = True
columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without dnssec by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without DNSSEC (Public)', criteria=f'has_dnssec == False & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without DNSSEC (Private)', criteria=f'has_dnssec == False & {only_private}', columns_to_display=columns_to_display)
# Column creation with distribution of records with valid dnssec by region
criteria = 'has_dnssec == True & dnssec_is_valid == True'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With Valid DNSSEC (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With Valid DNSSEC (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)
# Column creation with distribution of records with invalid dnssec by region
criteria = 'has_dnssec == True & dnssec_is_valid == False'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With Invalid DNSSEC (Public)', criteria=f'{criteria} & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With Invalid DNSSEC (Private)', criteria=f'{criteria} & {only_private}', columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

,Region,Global #,Without DNSSEC (Public) #,Without DNSSEC (Public) %,Without DNSSEC (Private) #,Without DNSSEC (Private) %,With Valid DNSSEC (Public) #,With Valid DNSSEC (Public) %,With Valid DNSSEC (Private) #,With Valid DNSSEC (Private) %,With Invalid DNSSEC (Public) #,With Invalid DNSSEC (Public) %,With Invalid DNSSEC (Private) #,With Invalid DNSSEC (Private) %
0,California,2,0,0.000000,2,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
1,Florida,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
2,Georgia,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
3,Illinois,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
4,Michigan,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
5,Minnesota,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
6,New York,1,0,0.000000,1,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
7,Texas,2,0,0.000000,2,100.000000,0,0.0,0,0.0,0,0.0,0,0.0
8,Colorado,1,1,100.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0
9,Total,11,1,9.090909,10,90.909091,0,0.0,0,0.0,0,0.0,0,0.0


In [4]:
# Report in latex
report_results = get_extreme_values(analysis_df)

hei_public_without_dnssec = format(report_results.get("Total").get("Without DNSSEC (Public) %"), ".2f")
hei_public_with_valid_dnssec = format(report_results.get("Total").get("With Valid DNSSEC (Public) %"), ".2f")
hei_public_with_invalid_dnssec = format(report_results.get("Total").get("With Invalid DNSSEC (Public) %"), ".2f")

hei_private_without_dnssec = format(report_results.get("Total").get("Without DNSSEC (Private) %"), ".2f")
hei_private_with_valid_dnssec = format(report_results.get("Total").get("With Valid DNSSEC (Private) %"), ".2f")
hei_private_with_invalid_dnssec = format(report_results.get("Total").get("With Invalid DNSSEC (Private) %"), ".2f")

report_figure = """
\\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{charts/dnssec.pdf}
    \caption{Distribution of DNSSEC usage by regions.}\label{fig:dnssec}
\end{figure}
"""

report = f'{report_figure}\n\n'
report += f'The data presented in Figure~\\ref{{fig:dnssec}} provides an overview of the use of \gls{{dnssec}} at \glspl{{hei}} in \countryName. According to the data, {hei_public_without_dnssec}\% of the public institutions analyzed have not implemented \gls{{dnssec}} on their domains, while {hei_private_without_dnssec}\% of the private institutions analyzed also dont use \gls{{dnssec}}.\n\n'
report += f'On a positive note, {hei_public_with_valid_dnssec}\% of the public institutions analyzed have implemented \gls{{dnssec}} with a valid configurations in their domains, and {hei_private_with_valid_dnssec}\% of the private institutions have implemented valid DNSSEC.\n\n'
report += f'However, {hei_public_with_invalid_dnssec}\% of the public institutions analyzed have implemented invalid \gls{{dnssec}} in their domains, and {hei_private_with_invalid_dnssec}\% of the private institutions also have implemented \gls{{dnssec}} with invalid configurations.\n\n'

report += f'In terms of regional differences, private institutions in {report_results.get("With Valid DNSSEC (Private) %").get("top_regions")[0][0]} ({format(report_results.get("With Valid DNSSEC (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With Valid DNSSEC (Private) %").get("top_regions")[1][0]} ({format(report_results.get("With Valid DNSSEC (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With Valid DNSSEC (Private) %").get("top_regions")[2][0]} ({format(report_results.get("With Valid DNSSEC (Private) %").get("top_regions")[2][1], ".2f")}\%), '
report += f'and public institutions in {report_results.get("With Valid DNSSEC (Public) %").get("top_regions")[0][0]} ({format(report_results.get("With Valid DNSSEC (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With Valid DNSSEC (Public) %").get("top_regions")[1][0]} ({format(report_results.get("With Valid DNSSEC (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With Valid DNSSEC (Public) %").get("top_regions")[2][0]} ({format(report_results.get("With Valid DNSSEC (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{dnssec}}.\n\n'

report += f'In contrast, private institutions in {report_results.get("With Valid DNSSEC (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("With Valid DNSSEC (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With Valid DNSSEC (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("With Valid DNSSEC (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With Valid DNSSEC (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("With Valid DNSSEC (Private) %").get("bottom_regions")[2][1], ".2f")}\%), '
report += f'and public institutions in {report_results.get("With Valid DNSSEC (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("With Valid DNSSEC (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With Valid DNSSEC (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("With Valid DNSSEC (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With Valid DNSSEC (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("With Valid DNSSEC (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of \gls{{dnssec}}.\n\n'

report += f'Finally, when it comes to invalid settings, private institutions in {report_results.get("With Invalid DNSSEC (Private) %").get("top_regions")[0][0]} ({format(report_results.get("With Invalid DNSSEC (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With Invalid DNSSEC (Private) %").get("top_regions")[1][0]} ({format(report_results.get("With Invalid DNSSEC (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With Invalid DNSSEC (Private) %").get("top_regions")[2][0]} ({format(report_results.get("With Invalid DNSSEC (Private) %").get("top_regions")[2][1], ".2f")}\%), '
report += f'and public institutions in {report_results.get("With Invalid DNSSEC (Public) %").get("top_regions")[0][0]} ({format(report_results.get("With Invalid DNSSEC (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With Invalid DNSSEC (Public) %").get("top_regions")[1][0]} ({format(report_results.get("With Invalid DNSSEC (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With Invalid DNSSEC (Public) %").get("top_regions")[2][0]} ({format(report_results.get("With Invalid DNSSEC (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{dnssec}} with invalid configurations.\n\n'

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)


\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/dnssec.pdf}
    \caption{Distribution of DNSSEC usage by regions.}\label{fig:dnssec}
\end{figure}


The data presented in Figure~\ref{fig:dnssec} provides an overview of the use of \gls{dnssec} at \glspl{hei} in \countryName. According to the data, 9.09\% of the public institutions analyzed have not implemented \gls{dnssec} on their domains, while 90.91\% of the private institutions analyzed also dont use \gls{dnssec}.

On a positive note, 0.00\% of the public institutions analyzed have implemented \gls{dnssec} with a valid configurations in their domains, and 0.00\% of the private institutions have implemented valid DNSSEC.

However, 0.00\% of the public institutions analyzed have implemented invalid \gls{dnssec} in their domains, and 0.00\% of the private institutions also have implemented \gls{dnssec} with invalid configurations.

In terms of regional differences, private institutions in Californi